# Step 12 - Final Case Study Questions

To finish up our entire cryptocurrency case study - let's now calculate exactly the same query that we just created for Nandita's data with our entire dataset with all mentors included!

## Create The Base Table
Create a summary table `mentor_performance` which includes the following metrics for each member and ticker:

- Count of purchase transactions
- Initial investment
- Purchase fees
- Dollar cost average of purchases
- Count of sales transactions
- Gross revenue amount
- Sales fees
- Average selling price
- Actual Profitability (final portfolio value + gross sales revenue - purchase fees - sales fees) / initial investment amount
- Theoretical Profitability (final portfolio value with no sales - purchase fees) / initial investment amount

In [1]:
# Importing packages:
import pandas as pd
import os
import sqlalchemy

# Connecting to MySQL database:
host = os.environ.get('mysql_host')
user = os.environ.get('mysql_user')
password = os.environ.get('mysql_password')
engine = sqlalchemy.create_engine(f'mysql+pymysql://{user}:{password}@{host}/trading')

In [7]:
query = """
WITH cte_portfolio AS (
  SELECT
    members.first_name,
    members.region,
    transactions.ticker,
    transactions.txn_type,
    COUNT(*) AS transaction_count,
    SUM(transactions.quantity) AS total_quantity,
    SUM(transactions.quantity * prices.price) AS gross_values,
    SUM(transactions.quantity * prices.price * transactions.percentage_fee / 100) AS fees 
  FROM trading.transactions
  INNER JOIN trading.members
    ON transactions.member_id = members.member_id
  INNER JOIN trading.prices
    ON transactions.ticker = prices.ticker
    AND transactions.txn_date = prices.market_date
  GROUP BY
    members.first_name,
    members.region,
    transactions.ticker,
    transactions.txn_type
),
cte_summary AS (
  SELECT
    first_name,
    region,
    ticker,
    SUM(
      CASE
        WHEN txn_type = 'BUY' THEN total_quantity
        WHEN txn_type = 'SELL' THEN -total_quantity
      END
    ) AS final_quantity,
    SUM(CASE WHEN txn_type = 'BUY' THEN gross_values ELSE 0 END) AS initial_investment,
    SUM(CASE WHEN txn_type = 'SELL' THEN gross_values ELSE 0 END) AS sales_revenue,
    SUM(CASE WHEN txn_type = 'BUY' THEN fees ELSE 0 END) AS purchase_fees,
    SUM(CASE WHEN txn_type = 'SELL' THEN fees ELSE 0 END) AS sales_fees,
    SUM(CASE WHEN txn_type = 'BUY' THEN total_quantity ELSE 0 END) AS purchase_quantity,
    SUM(CASE WHEN txn_type = 'SELL' THEN total_quantity ELSE 0 END) AS sales_quantity,
    SUM(CASE WHEN txn_type = 'BUY' THEN transaction_count ELSE 0 END) AS purchase_transactions,
    SUM(CASE WHEN txn_type = 'SELL' THEN transaction_count ELSE 0 END) AS sales_transactions
  FROM cte_portfolio
  GROUP BY
    first_name,
    region,
    ticker
),
cte_metrics AS (
  SELECT
    summary.first_name,
    summary.region,
    summary.ticker,
    summary.final_quantity * final.price AS actual_final_value,
    summary.purchase_quantity * final.price AS theoretical_final_value,
    summary.sales_revenue,
    summary.purchase_fees,
    summary.sales_fees,
    summary.initial_investment,
    summary.purchase_quantity,
    summary.sales_quantity,
    summary.purchase_transactions,
    summary.sales_transactions,
    summary.initial_investment / purchase_quantity AS dollar_cost_average,
    summary.sales_revenue / sales_quantity AS average_selling_price
  FROM cte_summary AS summary
  INNER JOIN trading.prices AS final
    ON summary.ticker = final.ticker
  WHERE final.market_date = '2021-08-29'
)
SELECT
  first_name,
  region,
  ticker,
  actual_final_value AS final_portfolio_value,
  ( actual_final_value + sales_revenue - purchase_fees - sales_fees ) / initial_investment AS actual_profitability,
  ( theoretical_final_value - purchase_fees ) / initial_investment AS theoretical_profitability,
  dollar_cost_average,
  average_selling_price,
  sales_revenue,
  purchase_fees,
  sales_fees,
  initial_investment,
  purchase_quantity,
  sales_quantity,
  purchase_transactions,
  sales_transactions
FROM cte_metrics;
"""

mentor_performance = pd.read_sql_query(query, engine)
mentor_performance.to_sql(
  name='mentor_performance',
  con=engine,
  index=False,
  if_exists='replace',
)

In [8]:
pd.read_sql_query(
    """
    SELECT *
    FROM mentor_performance
    """,
    engine
)

,first_name,region,ticker,final_portfolio_value,actual_profitability,theoretical_profitability,dollar_cost_average,average_selling_price,sales_revenue,purchase_fees,sales_fees,initial_investment,purchase_quantity,sales_quantity,purchase_transactions,sales_transactions
0,Vipul,United States,ETH,1.310256e+07,4.672466,5.314622,597.667322,626.242026,4.567939e+05,7104.570101,1188.379519,2.900194e+06,4852.521567,729.420717,991.0,149.0
1,Alex,United States,ETH,8.166075e+06,5.160549,6.195711,512.700544,594.103480,3.943560e+05,4190.281069,1055.122692,1.657805e+06,3233.476848,663.783429,651.0,136.0
2,Abe,United States,ETH,6.639150e+06,6.306367,6.966968,455.956219,402.869593,1.024365e+05,2830.408924,280.268725,1.068519e+06,2343.469084,254.267181,452.0,49.0
3,Danny,Australia,ETH,1.113844e+07,5.082495,6.230089,509.872366,566.702622,5.732548e+05,5824.685434,1455.005348,2.302888e+06,4516.597244,1011.561933,904.0,200.0
4,Sonia,Australia,ETH,1.132069e+07,4.478792,5.292087,600.197716,547.838968,4.445042e+05,6743.608509,1074.114552,2.625122e+06,4373.762105,811.377514,864.0,161.0
5,Ben,Australia,ETH,1.342841e+07,5.566492,5.798154,547.840959,575.241107,1.245384e+05,6102.489005,311.466406,2.433585e+06,4442.136852,216.497826,877.0,44.0
6,Nandita,United States,ETH,1.113479e+07,4.941266,5.308057,598.397258,542.484897,1.725919e+05,5783.326972,447.938122,2.287097e+06,3822.037200,318.150636,756.0,70.0
7,Leah,Asia,ETH,5.011671e+06,4.270687,5.483030,579.309227,661.522994,4.035602e+05,3216.060293,986.188546,1.267016e+06,2187.115438,610.047063,435.0,126.0
8,Charlie,United States,ETH,5.655596e+06,4.384908,5.823239,545.483350,636.245975,5.052154e+05,3500.084034,1162.340573,1.403940e+06,2573.754758,794.056716,501.0,158.0
9,Enoch,Africa,ETH,2.183933e+06,2.927689,5.826837,545.139758,571.750353,6.045780e+05,2441.665069,1602.260204,9.510804e+05,1744.654252,1057.415964,351.0,209.0


**Question 1**

Which mentors have the greatest actual profitability for each ticker?

I answered this question with this twisted query

In [19]:
pd.read_sql_query(
    """
    WITH eth_max_profitability AS (
      SELECT
        first_name,
        ticker,
        MAX(actual_profitability) AS actual_profitability
      FROM mentor_performance
      WHERE ticker='ETH'
      GROUP BY first_name, ticker
      ORDER BY actual_profitability DESC
      LIMIT 1
    ),

    btc_max_profitability AS (
      SELECT
        first_name,
        ticker,
        MAX(actual_profitability) AS actual_profitability
      FROM mentor_performance
      WHERE ticker='BTC'
      GROUP BY first_name, ticker
      ORDER BY actual_profitability DESC
      LIMIT 1
    )  
    SELECT *
    FROM btc_max_profitability
    UNION
    SELECT *
    FROM eth_max_profitability;
    """,
    engine
)

,first_name,ticker,actual_profitability
0,Charlie,BTC,3.725216
1,Abe,ETH,6.306367


The following one is better:

In [21]:
pd.read_sql_query(
    """
    WITH cte_rank AS (
        SELECT
          first_name,
          ticker,
          actual_profitability,
          RANK() OVER (PARTITION BY ticker ORDER BY actual_profitability DESC) AS profitability_rank
        FROM mentor_performance
    )
    SELECT *
    FROM cte_rank
    WHERE profitability_rank=1;
    """,
    engine
)

,first_name,ticker,actual_profitability,profitability_rank
0,Charlie,BTC,3.725216,1
1,Abe,ETH,6.306367,1


**Question 2**

Which mentors have the greatest difference in actual vs theoretical profitability for each ticker?

In [30]:
pd.read_sql_query(
    """
    WITH cte_rank AS (
        SELECT
          first_name,
          ticker,
          ABS(actual_profitability - theoretical_profitability) AS profitability_delta,
          RANK() OVER (
            PARTITION BY ticker ORDER BY ABS(actual_profitability - theoretical_profitability) DESC
            ) AS profitability_delta_rank
        FROM mentor_performance
    )
    SELECT 
      *
    FROM cte_rank
    WHERE profitability_delta_rank=1;
    """,
    engine
)

,first_name,ticker,profitability_delta,profitability_delta_rank
0,Pavan,BTC,1.232768,1
1,Ayush,ETH,3.129217,1


**Question 3**

What is the total amount of sales revenue made by all mentors for each region? (combined BTC and ETH)

In [34]:
pd.read_sql_query(
    """
    SELECT
      region,
      SUM(sales_revenue) AS total_sales_revenue
    FROM mentor_performance
    GROUP BY region
    ORDER BY total_sales_revenue DESC;
    """,
    engine
)

,region,total_sales_revenue
0,United States,6.639637e+07
1,Australia,4.243766e+07
2,Asia,1.357058e+07
3,Africa,9.598209e+06
4,India,7.234158e+06


**Question 4**

What is the average actual profitability for each region for each ticker?

In [38]:
pd.read_sql_query(
    """
    SELECT
      region,
      ticker,
      AVG(actual_profitability) AS avg_actual_profitability
    FROM mentor_performance
    GROUP BY region, ticker
    ORDER BY ticker, avg_actual_profitability DESC;
    """,
    engine
)

,region,ticker,avg_actual_profitability
0,India,BTC,3.527151
1,Africa,BTC,3.322104
2,United States,BTC,3.310800
3,Australia,BTC,3.229264
4,Asia,BTC,3.207139
5,United States,ETH,4.746694
6,Australia,ETH,4.745562
7,India,ETH,4.311813
8,Asia,ETH,4.270687
9,Africa,ETH,2.927689


**Question 5**

Which mentors have the largest initial investment in each ticker?

In [45]:
pd.read_sql_query(
    """
    WITH cte_rank AS (
        SELECT
          first_name,
          ticker,
          initial_investment,
          RANK() OVER (
              PARTITION BY ticker ORDER BY initial_investment DESC
          ) AS investment_rank
        FROM mentor_performance
    )
    SELECT *
    FROM cte_rank
    WHERE investment_rank=1;
    """,
    engine
)

,first_name,ticker,initial_investment,investment_rank
0,Leah,BTC,6.492218e+07,1
1,Vipul,ETH,2.900194e+06,1


# References

- [Data With Danny Course - Step 12](https://github.com/DataWithDanny/sql-masterclass/blob/main/course-content/step12.md)